# Section 1 Install some stuff an load data
*PLACEHOLDER*

In [3]:
# Install dependencies as needed:
# Install dependencies based on OS
import os
import sys
if sys.platform == "win32":
    %pip install kagglehub[pandas-datasets] plotly
else:
    %pip install "kagglehub[pandas-datasets]" "plotly"

import kagglehub
from kagglehub import KaggleDatasetAdapter
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd


Note: you may need to restart the kernel to use updated packages.


/home/janis/miniconda3/envs/general/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Set the path to the file you'd like to load
file_paths = ["high_popularity_spotify_data.csv", "low_popularity_spotify_data.csv"]
database = []

# Load the latest version
for file_path in file_paths:
    df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "solomonameh/spotify-music-dataset",
    file_path,
    # Provide any additional arguments like 
    # sql_query or pandas_kwargs. See the 
    # documenation for more information:
    # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
    )
    database.append(df)

print("First 5 records:", df.head())

/tmp/ipykernel_4953/3214104888.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(
/tmp/ipykernel_4953/3214104888.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:    time_signature  track_popularity  speechiness  danceability  playlist_name  \
0             4.0                23       0.0393         0.636  Rock Classics   
1             4.0                53       0.0317         0.572  Rock Classics   
2             4.0                55       0.0454         0.591  Rock Classics   
3             4.0                64       0.1010         0.443  Jazz Classics   
4             4.0                62       0.0298         0.685  Jazz Classics   

                       track_artist  duration_ms  energy playlist_genre  \
0      Creedence Clearwater Revival     138053.0   0.746           rock   
1                         Van Halen     241600.0   0.835           rock   
2                      Stevie Nicks     329413.0   0.804           rock   
3  Ella Fitzgerald, Louis Armstrong     185160.0   0.104           jazz   
4                    Galt MacDermot     205720.0   0.472           jazz   

  playlist_subgenre  ...          track_album

# Some Data wrangling I guess

Lil section for preprocessing although I really dont think we need it. But I also havent tried yet so maybe we do


ToDo:
Need the date column in a proper format



In [20]:
#Combine Into one
spotify_combined = pd.concat(database, ignore_index=True)
print("Combined DataFrame shape:", spotify_combined.shape)

#Convert that to dates
spotify_combined["track_album_release_date"] = pd.to_datetime(spotify_combined["track_album_release_date"], format = "mixed")



Combined DataFrame shape: (4831, 29)


# Figure 1
I dont know yet what figure 1 Is but ill figure it out.

In [ ]:
spotify_combined["play"]

In [24]:
# Most produced Genres per decade as barplot
# Display change from one decade to previous and next by grouping decades in trios 
# differnetiate with colors
# repeat for popular songs to get most popular genres per decade
spotify_combined['year'] = spotify_combined['track_album_release_date'].dt.year
spotify_combined['decade'] = (spotify_combined['year'] // 10) * 10
decade_genre_counts = spotify_combined.groupby(['decade', 'playlist_genre']).size().reset_index(name='count')

fig = px.bar(decade_genre_counts, x='decade', y='count', color='playlist_genre',
             title='Most Produced Genres per Decade',
             labels={'decade': 'Decade', 'count': 'Number of Songs', 'playlist_genre': 'Genre'},
             barmode='group')
fig.show()
